In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.environ['KAGGLE_USERNAME'] = "lampe01" # username from the json file
os.environ['KAGGLE_KEY'] = "0fe21bb583c2cd3230e9d928c96e27f7" # key from the json file
os.environ['KAGGLE_CONFIG_DIR'] = "/content/drive/MyDrive/CirCor"

In [3]:
pip install pydub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
pip install ffmpeg

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for ffmpeg: filename=ffmpeg-1.4-py3-none-any.whl size=6084 sha256=7b70cf2348f46eb80293cf19e3c5c8519bb168482d0b4606aa6a8520d1c4548d
  Stored in directory: /root/.cache/pip/wheels/64/80/6e/caa3e16deb0267c3cbfd36862058a724144e19fdb9eb03af0f
Successfully built ffmpeg


In [5]:
pip install Ipython

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 6.6 MB/s 


In [6]:
pip uninstall scikit-learn

Found existing installation: scikit-learn 1.0.2
Uninstalling scikit-learn-1.0.2:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/scikit_learn-1.0.2.dist-info/*
    /usr/local/lib/python3.7/dist-packages/scikit_learn.libs/libgomp-a34b3233.so.1.0.0
    /usr/local/lib/python3.7/dist-packages/sklearn/*
Proceed (y/n)? y
  Successfully uninstalled scikit-learn-1.0.2


In [7]:
pip install scikit-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 24.8 MB 83.4 MB/s 


In [8]:
pip install librosa

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [9]:
pip install pyAudioProcessing

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.0 MB 6.9 MB/s 
     |████████████████████████████████| 25.9 MB 1.4 MB/s 
     |████████████████████████████████| 280 kB 43.4 MB/s 
     |████████████████████████████████| 11.5 MB 20.4 MB/s 
     |████████████████████████████████| 22.3 MB 84.2 MB/s 
  Attempting uninstall: scipy
    Found existing installation: scipy 1.7.3
    Uninstalling scipy-1.7.3:
      Successfully uninstalled scipy-1.7.3
  Attempting uninstall: pluggy
    Found existing installation: pluggy 0.7.1
    Uninstalling pluggy-0.7.1:
      Successfully uninstalled pluggy-0.7.1
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.0.2
    Uninstalling scikit-learn-1.0.2:
      Successfully uninstalled scikit-learn-1.0.2
  Attempting uninstall: pytest
    Found existing installation: pytest 3.6.4
    Uninstalling pytest-3.6.4:
      Successfully uninsta

In [10]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as plt
import itertools
import os
import sys
import pywt
import librosa
import sklearn
import scipy
import scipy.io
import scipy.io.wavfile
from scipy import signal
from scipy.stats import kurtosis, skew, tstd
from scipy.signal import periodogram, welch, butter, lfilter, freqz
from os import path
from pydub import AudioSegment
from collections import defaultdict
import tensorflow as tf
from tensorflow import keras
from keras import optimizers
from keras.models import Model, load_model, Sequential
from keras.layers import Input, Dense, Dropout, AlphaDropout, Activation, Flatten, LSTM
from keras.callbacks import ModelCheckpoint, TensorBoard,EarlyStopping,ReduceLROnPlateau,ProgbarLogger
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, precision_recall_curve, recall_score, classification_report, auc, roc_curve, precision_recall_fscore_support, f1_score, accuracy_score
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from keras.utils import np_utils
from keras.wrappers.scikit_learn import KerasClassifier
from pyAudioProcessing.extract_features import get_features

In [11]:
mdata = pd.read_csv('/content/drive/MyDrive/ReCirCor/training_data.csv')
input_dir = '/content/drive/MyDrive/ReCirCor/training_data/training_data'

In [12]:
#Gets all the txt files
def getfilenames(inputdir):
    Fname = list()
    Fname = sorted([fname for fname in os.listdir(inputdir) if not fname.startswith('.')and fname.endswith('.txt')])
    
    return Fname

In [13]:
#List of Labels - 0 for normal, 1 for murmur
labels = [0,0,0,0,0,1,1,1,1,1,1,1,1,0,1,1,1,0,0,0,1,1,1]
len(labels)

23

In [14]:
#They were originally named differently and I have renamed and numbered them for ease of access.
#The michigan heart sound recordings are in mp3 format
#The recordings are converted to wav format for easy processing as most python libraries require the audio format to be wav.

def converttowav(filepath):
  destinationpath = filepath.split('.')[0] + '.wav'
  sound = AudioSegment.from_mp3(filepath)
  sound.export(destinationpath, format = "wav")

filepathslist = list()
for i in range(23):
  filepath = str(i+1) + '.mp3'
  converttowav(filepath)
  filepathslist.append(str(i+1)+'.wav')

In [15]:
#Creating an empty dataframe with columns to store filenames and the respective labels
dataframe = pd.DataFrame(columns = ['filename', 'labels'])

In [16]:
filepathslist

['1.wav',
 '2.wav',
 '3.wav',
 '4.wav',
 '5.wav',
 '6.wav',
 '7.wav',
 '8.wav',
 '9.wav',
 '10.wav',
 '11.wav',
 '12.wav',
 '13.wav',
 '14.wav',
 '15.wav',
 '16.wav',
 '17.wav',
 '18.wav',
 '19.wav',
 '20.wav',
 '21.wav',
 '22.wav',
 '23.wav']

In [17]:
labels

[0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1]

In [18]:
labels

[0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1]

In [19]:
#Inserting filename and label values into dataframe
dataframe['filename'] = filepathslist
dataframe['labels'] = labels

In [20]:
dataframenew = pd.DataFrame(columns = ['new ID', 'wav filename', 'flag value', 'labels'])

In [21]:
import random

In [22]:
newID = list()
wavfilename = list()
newlabels = list()
flagvalue = list()
for i in range(23):
  #x = random.randint(1,23)
  newID.append(str(i+1))
  wavfilename.append(str(i+1)+'.wav')
  flagvalue.append(0)
  newlabels.append(labels[i])

In [23]:
#Get only murmur recordings
def getrecmur(inputdir):
    fname = getfilenames(inputdir)
    items = []
    rinfo = []
    recf = []
    labels = []
    recfile = []
    for i in fname:
        i = inputdir + '/' + i
        with open(i, 'r') as f:
            content = f.readlines()
        numloc = int(content[0].split(' ')[1])
        for i in range(numloc):
            for p in range(len(content)):
                t = content[p].split('\n')
                for it in t:
                    if it.startswith('#Murmur:'):
                        tar = it.split(': ')[1]
                        if tar == 'Present':
                          recinfo = content[i+1].split(' ')
                          rinfo.append(recinfo)
                          l = 1
                          labels.append(l)
                        elif tar == 'Absent':
                          recinfo = content[i+1].split(' ')
                          rinfo.append(recinfo)
                          l = 0
                          labels.append(l)
    for items in rinfo:
        for i in range(len(items)):
            if items[i].endswith('.wav'):
                recfile = input_dir + '/' + items[i]
        recf.append(recfile)
    return rinfo, recf, labels

rin, ref, label = getrecmur(input_dir)

In [24]:
len(rin)

3007

In [25]:
x = 23
for i in range(1000):
  x = x + 1
  wavfilename.append(ref[i])
  newlabels.append(label[i])
  flagvalue.append(1)
  newID.append(x)

In [26]:
len(newID)

1023

In [27]:
dataframenew['new ID'] = newID
dataframenew['wav filename'] = wavfilename
dataframenew['flag value'] = flagvalue
dataframenew['labels'] = newlabels
dataframenew

,new ID,wav filename,flag value,labels
0,1,1.wav,0,0
1,2,2.wav,0,0
2,3,3.wav,0,0
3,4,4.wav,0,0
4,5,5.wav,0,0
...,...,...,...,...
1018,1019,/content/drive/MyDrive/ReCirCor/training_data/...,1,0
1019,1020,/content/drive/MyDrive/ReCirCor/training_data/...,1,0
1020,1021,/content/drive/MyDrive/ReCirCor/training_data/...,1,0
1021,1022,/content/drive/MyDrive/ReCirCor/training_data/...,1,0


In [28]:
dataframe

,filename,labels
0,1.wav,0
1,2.wav,0
2,3.wav,0
3,4.wav,0
4,5.wav,0
5,6.wav,1
6,7.wav,1
7,8.wav,1
8,9.wav,1
9,10.wav,1


In [29]:
dataframenew['labels'].value_counts()

0    786
1    237
Name: labels, dtype: int64

In [30]:
X_train, X_test, y_Train, y_Test = train_test_split(dataframenew['new ID'], dataframenew['labels'], train_size=0.8, shuffle=True)

In [31]:
X_train

705    706
285    286
531    532
534    535
903    904
      ... 
731    732
919    920
559    560
733    734
877    878
Name: new ID, Length: 818, dtype: object

In [32]:
X_test

419    420
585    586
63      64
104    105
161    162
      ... 
708    709
755    756
505    506
702    703
288    289
Name: new ID, Length: 205, dtype: object

In [33]:
#Computing Features - 

#Discrete Wavelet Transform
def computeDWTs(x):
    #approximation and detail coefficients
    res = np.array([])
    ca, cd = pywt.dwt(x, 'db8')
    #Compute mean, stdev, skew, kurtosis
    camean = np.mean(ca)
    res = np.hstack((res,camean))
    castdev = tstd(ca)
    res = np.hstack((res,castdev))
    caskew = skew(ca)
    res = np.hstack((res,caskew))
    cakurtosis = kurtosis(ca)
    res = np.hstack((res,cakurtosis))
    cdmean = np.mean(cd)
    res = np.hstack((res,cdmean))
    cdstdev = tstd(cd)
    res = np.hstack((res,cdstdev))
    cdskew = skew(cd)
    res = np.hstack((res,cdskew))
    cdkurtosis = kurtosis(cd)
    res = np.hstack((res, cdkurtosis))
    return res

#Computing the Spectral Entropy of the signal - Shannon Entropy of the Power Spectral Density (PSD) of the data
def SpectralEntropy(Signal, SampleRate, method = 'fft', nperseg = None, normalize = False):
    Signal = np.array(Signal)
    if method == 'fft':
      _, psd = periodogram(Signal, SampleRate)
    elif method == 'welch':
      _, psd = welch(Signal, SampleRate, nperseg=nperseg)
    psd_norm = np.divide(psd, psd.sum())
    se = -np.multiply(psd_norm, np.log2(psd_norm)).sum()
    if normalize:
        se /= np.log2(psd_norm.size)
    return se

#Computing the Power Spectral Density(PSD) of signal - 
def computepsd(x):
    freq, psd = signal.welch(x)
    return psd

#Computing Mel-frequency cepstral coefficients, Gammatone frequency cepstral coefficients, RMS Energy, Zero-crossing rate, mean, variance, skew, and kurtosis of the signal
def computeMFCC(x, sr):
    mfccs = np.mean(librosa.feature.mfcc(y = x, sr = sr, n_mfcc = 13).T, axis = 0)
    return mfccs

def computeGFCC(file_name):
    gfcc = get_features(file=file_name, feature_names=["gfcc"])
    gfcc = np.array(gfcc['audio'][file_name]['features'])
    return gfcc

def computeRMSEn(x):
    rmsen = np.mean(librosa.feature.rms(x, frame_length=2048, hop_length=1024, center=True).T, axis = 0)
    return rmsen

def computeZCR(x):
    zcr = np.mean(librosa.feature.zero_crossing_rate(y = x, frame_length=2048, hop_length=1024).T, axis = 0)
    return zcr

def computestats(x):
#Mean, Variance, Skew, Kurtosis
    stats = []
    me = np.mean(x)
    stats.append(me)
    variance = np.var(x)
    stats.append(variance)
    sk = skew(x)
    stats.append(sk)
    kurt = kurtosis(x)
    stats.append(kurt)
    return stats

In [34]:
#Function to extract the required features
def extractfeatures(file_name, x, sr, *, mfcc: bool = False, dwt: bool = False, spectral_entropy: bool = False, psd: bool = False, gfcc: bool = False, rmse: bool = False, zcr: bool = False, stats: bool = False):
    finalfeatures = np.array([])
    if mfcc:
        mfccs = computeMFCC(x, sr)
        
        finalfeatures = np.hstack((finalfeatures, mfccs))
    if dwt:
        dwtr = computeDWTs(x)
        finalfeatures = np.hstack((finalfeatures, dwtr))
    if spectral_entropy:
        specen = SpectralEntropy(x, sr)
        finalfeatures = np.hstack((finalfeatures,specen))
    if psd:
        psds = computepsd(x)
        finalfeatures = np.hstack((finalfeatures, psds))
    if gfcc:
        gfccs = computeGFCC(file_name)
        finalfeatures = np.hstack((finalfeatures, gfccs))
    if rmse:
        rmses = computeRMSEn(x)
        finalfeatures = np.hstack((finalfeatures, rmses))
    if zcr:
        zcrs = computeZCR(x)
        finalfeatures = np.hstack((finalfeatures, zcrs))
    if stats:
        stats1 = computestats(x)
        finalfeatures = np.hstack((finalfeatures, stats1))
    return finalfeatures

In [35]:
#loads audio and extracts required features
def load_file_data(file_num, duration=15, sr = 4000):
    input_length=sr*duration
    notfeatures = []
    i = 0
    for file_no in file_num:
        try:
            #sound_file = folder + '/' + file_name
            sfile = int(file_no)
            sound_file = dataframenew['wav filename'][sfile-1]
            print ("load file ", i, sound_file)
            x, sr = librosa.load(sound_file,duration=duration,res_type='kaiser_fast')
            dur = librosa.get_duration(y=x, sr=sr)
            #Padding audio signal
            if (round(dur) < duration):
                print ("fixing audio length:", file_no, sound_file)
                y = librosa.util.fix_length(x, input_length)
            feature = extractfeatures(sound_file, x, sr, mfcc = True, dwt = True, psd = True, gfcc = True, rmse = True, zcr = True, stats = True)
            #dwt = True, psd = True, gfcc = True, rmse = True, zcr = True, stats = True
            notfeatures.append(feature)          
        except Exception as e:
            print(e)
            #notfeatures = np.asarray(features, dtype=np.float32)
        i = i+1 
    return notfeatures

In [36]:
x_train = []
x_test = []
x_train = load_file_data(X_train)
x_test = load_file_data(X_test)

Streaming output truncated to the last 5000 lines.
load file  313 /content/drive/MyDrive/ReCirCor/training_data/training_data/49999_AV.wav

        
 Extracting features gfcc 

        
Computing features for file 1 of 1 : /content/drive/MyDrive/ReCirCor/training_data/training_data/49999_AV.wav
load file  314 /content/drive/MyDrive/ReCirCor/training_data/training_data/49824_MV.wav

        
 Extracting features gfcc 

        
Computing features for file 1 of 1 : /content/drive/MyDrive/ReCirCor/training_data/training_data/49824_MV.wav
load file  315 /content/drive/MyDrive/ReCirCor/training_data/training_data/49897_MV.wav

        
 Extracting features gfcc 

        
Computing features for file 1 of 1 : /content/drive/MyDrive/ReCirCor/training_data/training_data/49897_MV.wav
load file  316 4.wav

        
 Extracting features gfcc 

        
Computing features for file 1 of 1 : 4.wav
load file  317 /content/drive/MyDrive/ReCirCor/training_data/training_data/50388_MV.wav

        
 Extr

In [37]:
x_train = np.asarray(x_train).astype(np.float32)
x_test = np.asarray(x_test).astype(np.float32)

In [38]:
scaler = MinMaxScaler()
scaler.fit(x_train)
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)

In [39]:
x_test.shape

(205, 200)

In [40]:
x_train.shape

(818, 200)

In [41]:
y_Train.dtype

dtype('int64')

In [42]:
y_Train = np.asarray(y_Train).astype(np.float32)

In [43]:
y_Test = np.asarray(y_Test).astype(np.float32)

In [44]:
traindataframe = pd.DataFrame(columns = ['x_train ID', 'labels'])
testdataframe = pd.DataFrame(columns = ['x_test ID', 'labels'])
traindataframe['x_train ID'] = X_train
testdataframe['x_test ID'] = X_test
traindataframe['labels'] = y_Train
testdataframe['labels'] = y_Test

In [45]:
testdataframe

,x_test ID,labels
419,420,0.0
585,586,0.0
63,64,0.0
104,105,1.0
161,162,0.0
...,...,...
708,709,0.0
755,756,0.0
505,506,0.0
702,703,0.0


In [46]:
xtrainwavfiles = list()
trainflagval = list()
for i in X_train:
  num = int(i)
  xtrainwavfiles.append(dataframenew['wav filename'][num-1])
  trainflagval.append(dataframenew['flag value'][num-1])

In [47]:
xtestwavfiles = list()
flagval = list()
for i in X_test:
  num = int(i)
  xtestwavfiles.append(dataframenew['wav filename'][num-1])
  flagval.append(dataframenew['flag value'][num-1])

In [48]:
pip install keras-tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 135 kB 6.3 MB/s 


In [49]:
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
from sklearn.feature_selection import SelectKBest, f_classif, chi2
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA, NMF
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt

In [50]:
pca = PCA(n_components=20)
x_train_pca = pca.fit_transform(x_train)
x_test_pca = pca.fit_transform(x_test)

In [51]:
x_train_pca.shape

(818, 20)

In [52]:
x_test_pca.shape

(205, 20)

In [53]:
dictofpca = defaultdict(list)

def getpcalist(x):
  for i in range(20):
    item = x[i]
    dictofpca[f"pca_{i}"].append(item)
  return dictofpca

for i in x_train_pca:
  pcanum = getpcalist(i)

testpcadict = defaultdict(list)

def getpcalist2(x):
  for i in range(20):
    item = x[i]
    testpcadict[f"pca_{i}"].append(item)
  return testpcadict

for i in x_test_pca:
  tpca = getpcalist2(i)

In [54]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.datasets import make_classification
clf = AdaBoostClassifier(n_estimators=100, random_state=0)
clf.fit(x_train_pca, y_Train)

AdaBoostClassifier(n_estimators=100, random_state=0)

In [55]:
clf.score(x_test_pca, y_Test)

0.7121951219512195

In [56]:
y_pred1 = clf.predict(x_test_pca)
print("Classification Report : \n\n", classification_report(y_Test, y_pred1))

Classification Report : 

               precision    recall  f1-score   support

         0.0       0.79      0.86      0.82       158
         1.0       0.31      0.21      0.25        47

    accuracy                           0.71       205
   macro avg       0.55      0.54      0.54       205
weighted avg       0.68      0.71      0.69       205



In [57]:
df1 = pd.DataFrame({'Input':X_test, 'Wav filename':xtestwavfiles, 'flag value':flagval, 'Real Values':y_Test, 'Predicted Values':y_pred1})
df1

,Input,Wav filename,flag value,Real Values,Predicted Values
419,420,/content/drive/MyDrive/ReCirCor/training_data/...,1,0.0,0.0
585,586,/content/drive/MyDrive/ReCirCor/training_data/...,1,0.0,0.0
63,64,/content/drive/MyDrive/ReCirCor/training_data/...,1,0.0,1.0
104,105,/content/drive/MyDrive/ReCirCor/training_data/...,1,1.0,0.0
161,162,/content/drive/MyDrive/ReCirCor/training_data/...,1,0.0,0.0
...,...,...,...,...,...
708,709,/content/drive/MyDrive/ReCirCor/training_data/...,1,0.0,0.0
755,756,/content/drive/MyDrive/ReCirCor/training_data/...,1,0.0,0.0
505,506,/content/drive/MyDrive/ReCirCor/training_data/...,1,0.0,0.0
702,703,/content/drive/MyDrive/ReCirCor/training_data/...,1,0.0,1.0


In [58]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.svm import SVC

In [59]:
clf2 = LinearSVC(penalty='l2', loss='hinge')
clf2.fit(x_train_pca, y_Train)
clf2.score(x_test_pca, y_Test)

0.7414634146341463

In [60]:
y_pred2 = clf2.predict(x_test_pca)
print("Classification Report : \n\n", classification_report(y_Test, y_pred2))

Classification Report : 

               precision    recall  f1-score   support

         0.0       0.77      0.94      0.85       158
         1.0       0.25      0.06      0.10        47

    accuracy                           0.74       205
   macro avg       0.51      0.50      0.48       205
weighted avg       0.65      0.74      0.68       205



In [61]:
df2 = pd.DataFrame({'Input':X_test, 'Wav filename':xtestwavfiles, 'flag value': flagval, 'Real Values':y_Test, 'Predicted Values':y_pred2})
df2

,Input,Wav filename,flag value,Real Values,Predicted Values
419,420,/content/drive/MyDrive/ReCirCor/training_data/...,1,0.0,0.0
585,586,/content/drive/MyDrive/ReCirCor/training_data/...,1,0.0,0.0
63,64,/content/drive/MyDrive/ReCirCor/training_data/...,1,0.0,0.0
104,105,/content/drive/MyDrive/ReCirCor/training_data/...,1,1.0,0.0
161,162,/content/drive/MyDrive/ReCirCor/training_data/...,1,0.0,0.0
...,...,...,...,...,...
708,709,/content/drive/MyDrive/ReCirCor/training_data/...,1,0.0,0.0
755,756,/content/drive/MyDrive/ReCirCor/training_data/...,1,0.0,0.0
505,506,/content/drive/MyDrive/ReCirCor/training_data/...,1,0.0,0.0
702,703,/content/drive/MyDrive/ReCirCor/training_data/...,1,0.0,0.0


In [62]:
clf4 = SGDClassifier(loss = 'hinge').fit(x_train_pca, y_Train)

In [63]:
clf4.score(x_test_pca, y_Test)

0.6487804878048781

In [64]:
y_pred4 = clf4.predict(x_test_pca)
#y_pred=np.argmax(y_pred, axis=1)
print("Classification Report : \n\n", classification_report(y_Test, y_pred4))

Classification Report : 

               precision    recall  f1-score   support

         0.0       0.80      0.73      0.76       158
         1.0       0.30      0.38      0.33        47

    accuracy                           0.65       205
   macro avg       0.55      0.56      0.55       205
weighted avg       0.68      0.65      0.66       205



In [65]:
df4 = pd.DataFrame({'Input':X_test, 'Wav filename':xtestwavfiles, 'flag value': flagval, 'Real Values':y_Test, 'Predicted Values':y_pred4})
df4

,Input,Wav filename,flag value,Real Values,Predicted Values
419,420,/content/drive/MyDrive/ReCirCor/training_data/...,1,0.0,0.0
585,586,/content/drive/MyDrive/ReCirCor/training_data/...,1,0.0,0.0
63,64,/content/drive/MyDrive/ReCirCor/training_data/...,1,0.0,0.0
104,105,/content/drive/MyDrive/ReCirCor/training_data/...,1,1.0,1.0
161,162,/content/drive/MyDrive/ReCirCor/training_data/...,1,0.0,0.0
...,...,...,...,...,...
708,709,/content/drive/MyDrive/ReCirCor/training_data/...,1,0.0,0.0
755,756,/content/drive/MyDrive/ReCirCor/training_data/...,1,0.0,0.0
505,506,/content/drive/MyDrive/ReCirCor/training_data/...,1,0.0,1.0
702,703,/content/drive/MyDrive/ReCirCor/training_data/...,1,0.0,1.0


In [67]:
param_grid = {'C': [0.1, 0.5, 1, 10,30, 40, 50, 75, 100, 500, 1000],
              #0.01, 0.1, 0.5, 1, 10, 30, 40, 50, 75, 100, 500, 1000
              'gamma': [0.01, 0.05, 0.07, 0.1, 0.5, 1, 5, 10, 50],
              #0.01, 0.05, 0.07, 0.1, 0.5, 1, 0.75, 0.25, 0.001, 5, 10, 50
              'kernel': ['rbf', 'poly', 'linear']}

grid = GridSearchCV(SVC(), param_grid, refit=True, verbose=1, cv=5)
grid.fit(x_train_pca, y_Train)
best_params = grid.best_params_
print(f"Best params: {best_params}")

svmwpca = SVC(**best_params)
svmwpca.fit(x_train_pca, y_Train)

Fitting 5 folds for each of 432 candidates, totalling 2160 fits
Best params: {'C': 10, 'gamma': 0.75, 'kernel': 'rbf'}


SVC(C=10, gamma=0.75)

In [68]:
svmwpca.score(x_test_pca, y_Test)

0.6780487804878049

In [69]:
y_pred = svmwpca.predict(x_test_pca)
print(classification_report(y_Test, y_pred))

              precision    recall  f1-score   support

         0.0       0.77      0.84      0.80       158
         1.0       0.21      0.15      0.17        47

    accuracy                           0.68       205
   macro avg       0.49      0.49      0.49       205
weighted avg       0.64      0.68      0.66       205



In [70]:
pcadataframe = pd.DataFrame({'Input':X_train, 'Wav filename':xtrainwavfiles, 'flag value': trainflagval})
#'Labels':y_Train

In [71]:
for i in pcanum:
  pcadataframe[i] = pcanum[i]

In [72]:
y_newt = []
for i in y_Train:
  y_newt.append(int(i))

In [73]:
pcadataframe['Labels'] = y_newt

In [74]:
pcadataframe

,Input,Wav filename,flag value,pca_0,pca_1,pca_2,pca_3,pca_4,pca_5,pca_6,...,pca_11,pca_12,pca_13,pca_14,pca_15,pca_16,pca_17,pca_18,pca_19,Labels
705,706,/content/drive/MyDrive/ReCirCor/training_data/...,1,-0.580409,0.086189,0.269721,-0.304425,-0.275187,-0.381815,0.025136,...,-0.145661,0.009350,-0.127997,0.201993,0.140592,0.005553,-0.065124,-0.038208,0.037415,0
285,286,/content/drive/MyDrive/ReCirCor/training_data/...,1,-0.084916,-0.011565,0.448741,0.003800,0.139353,0.180470,-0.313552,...,-0.175974,0.414955,0.260061,0.041434,0.035619,0.101350,0.079966,-0.004793,-0.176857,1
531,532,/content/drive/MyDrive/ReCirCor/training_data/...,1,-0.344478,0.093229,0.471916,0.061435,-0.003288,-0.093233,0.008429,...,-0.020808,-0.257153,0.027224,-0.045267,0.017085,-0.029807,-0.050564,-0.004829,-0.008446,1
534,535,/content/drive/MyDrive/ReCirCor/training_data/...,1,-0.740799,0.208817,-0.312473,-0.101280,-0.398496,0.143896,0.039220,...,-0.071235,-0.008881,0.029332,0.047217,0.131854,-0.051440,0.014634,-0.106357,0.104023,0
903,904,/content/drive/MyDrive/ReCirCor/training_data/...,1,-0.881234,0.361992,0.489651,0.270753,0.123022,0.421335,-0.273834,...,0.606320,0.493655,-0.103523,-0.110915,-0.058742,0.096498,0.280978,0.151633,0.051181,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
731,732,/content/drive/MyDrive/ReCirCor/training_data/...,1,-0.279452,-0.002950,-0.164188,-0.145423,-0.273816,0.208583,0.006145,...,-0.205109,0.099046,-0.025873,-0.102164,-0.018781,0.087787,-0.076836,-0.028450,0.025767,0
919,920,/content/drive/MyDrive/ReCirCor/training_data/...,1,-0.479065,0.137557,-0.296747,-0.018401,-0.339547,0.125464,0.178425,...,-0.287443,-0.143492,-0.042005,0.154700,0.060244,-0.019356,0.012138,-0.010485,0.009601,0
559,560,/content/drive/MyDrive/ReCirCor/training_data/...,1,-0.775502,0.197593,-0.379451,-0.090369,-0.372044,-0.027952,0.311008,...,0.034496,0.018814,-0.117627,0.173522,0.208208,0.011164,0.003354,0.006000,0.045785,0
733,734,/content/drive/MyDrive/ReCirCor/training_data/...,1,-0.266024,0.014545,0.130251,-0.208413,0.079312,-0.226903,-0.195335,...,0.128788,-0.147782,0.133900,0.141004,0.036963,0.055183,0.053250,-0.075445,-0.086814,0


In [75]:
len(x_test_pca)

205

In [76]:
x_test_pca[0]

array([-0.16105396,  0.09043667, -0.44354942, -0.09097856, -0.07481404,
        0.03099684,  0.20099062, -0.01052068, -0.04011364, -0.02999514,
       -0.21445829,  0.11305723,  0.05986084,  0.02608867,  0.13730887,
       -0.05624208,  0.00809442, -0.02879396, -0.01896483,  0.00967637],
      dtype=float32)

In [77]:
testpcadict = defaultdict(list)
def getpcalist(x):
  for i in range(20):
    item = x[i]
    testpcadict[f"pca_{i}"].append(item)
  return testpcadict

In [78]:
for i in x_test_pca:
  tpca = getpcalist(i)

In [79]:
for i in tpca:
  print(i)

pca_0
pca_1
pca_2
pca_3
pca_4
pca_5
pca_6
pca_7
pca_8
pca_9
pca_10
pca_11
pca_12
pca_13
pca_14
pca_15
pca_16
pca_17
pca_18
pca_19


In [80]:
len(tpca['pca_0'])

205

In [81]:
pcaresult = pd.DataFrame({'Input':X_test, 'Wav filename':xtestwavfiles, 'flag value': flagval})

for i in tpca:
  pcaresult[i] = tpca[i]

y_newt2 = []
for i in y_Test:
  y_newt2.append(int(i))

pcaresult['Real Values'] = y_newt2
pcaresult['Predicted Values'] = y_pred

In [82]:
pcaresult

,Input,Wav filename,flag value,pca_0,pca_1,pca_2,pca_3,pca_4,pca_5,pca_6,...,pca_12,pca_13,pca_14,pca_15,pca_16,pca_17,pca_18,pca_19,Real Values,Predicted Values
419,420,/content/drive/MyDrive/ReCirCor/training_data/...,1,-0.161054,0.090437,-0.443549,-0.090979,-0.074814,0.030997,0.200991,...,0.059861,0.026089,0.137309,-0.056242,0.008094,-0.028794,-0.018965,0.009676,0,0.0
585,586,/content/drive/MyDrive/ReCirCor/training_data/...,1,-0.549992,-0.207378,-0.333647,0.170130,0.111977,0.101447,-0.205547,...,0.072368,-0.156120,-0.033906,-0.020126,-0.097380,0.025641,0.108757,0.039271,0,0.0
63,64,/content/drive/MyDrive/ReCirCor/training_data/...,1,0.487156,-0.193235,-0.034296,-0.550575,0.173546,-0.012375,-0.086222,...,0.125567,0.033496,-0.054161,-0.202684,0.110098,0.116485,-0.000115,0.008854,0,0.0
104,105,/content/drive/MyDrive/ReCirCor/training_data/...,1,0.570816,0.262524,-0.201789,-0.352261,0.088648,-0.219311,0.315034,...,-0.032481,0.193271,0.055908,-0.052307,0.023014,-0.014265,0.029397,-0.026272,1,0.0
161,162,/content/drive/MyDrive/ReCirCor/training_data/...,1,-0.144966,0.007696,0.132074,-0.266071,0.050279,-0.100092,0.090080,...,0.019417,0.018414,-0.039637,-0.042446,0.024283,0.012767,0.026220,-0.019144,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
708,709,/content/drive/MyDrive/ReCirCor/training_data/...,1,0.403718,-0.049178,-0.242210,-0.441089,0.227756,0.077438,-0.196146,...,0.130284,0.001622,-0.069486,-0.035662,0.169850,-0.095468,-0.061064,-0.099551,0,0.0
755,756,/content/drive/MyDrive/ReCirCor/training_data/...,1,-0.372802,0.081611,-0.003655,0.065640,0.132630,0.160864,0.016350,...,-0.097585,0.027729,-0.014981,-0.025239,0.036450,-0.017875,-0.023426,0.056479,0,0.0
505,506,/content/drive/MyDrive/ReCirCor/training_data/...,1,-0.285007,0.139399,0.473756,0.080842,-0.457965,-0.148621,0.117332,...,-0.050840,0.018162,0.115391,0.052712,-0.058024,-0.033098,0.038496,-0.026592,0,1.0
702,703,/content/drive/MyDrive/ReCirCor/training_data/...,1,0.414785,-0.264087,0.365869,-0.081206,0.541190,-0.254314,-0.201504,...,-0.046355,0.143295,0.014544,-0.276160,-0.025561,-0.134287,-0.152650,0.118237,0,1.0


In [83]:
pcadataframe.to_csv('traindf.csv')
pcaresult.to_csv('testresdf.csv')

In [89]:
from tqdm import tqdm_notebook as tqdm 
from sklearn.model_selection import GridSearchCV
import warnings
warnings.filterwarnings("ignore")

pipe_steps = [('scaler', MinMaxScaler()), ('pca', PCA()), ('SVM', SVC(kernel='rbf'))]

check_params= {
    'pca__n_components': [20], 
    'SVM__C': [0.1, 0.5, 1, 10,30, 40, 50, 75, 100, 500, 1000], 
    'SVM__gamma' : [0.01, 0.05, 0.07, 0.1, 0.5, 1, 5, 10, 50]
}

pipeline = Pipeline(pipe_steps)

for cv in tqdm(range(4,11)):
    create_grid = GridSearchCV(pipeline, param_grid=check_params, cv=cv)
    create_grid.fit(x_train, y_Train)
    print("score for %d fold CV -  %3.2f" %(cv, create_grid.score(x_test, y_Test)))
    print(" ")
    print(" ")
    print (create_grid.best_params_)

  0%|          | 0/7 [00:00<?, ?it/s]

score for 4 fold CV -  0.80
 
 
{'SVM__C': 30, 'SVM__gamma': 0.07, 'pca__n_components': 20}
score for 5 fold CV -  0.84
 
 
{'SVM__C': 1000, 'SVM__gamma': 0.05, 'pca__n_components': 20}
score for 6 fold CV -  0.82
 
 
{'SVM__C': 75, 'SVM__gamma': 0.1, 'pca__n_components': 20}
score for 7 fold CV -  0.82
 
 
{'SVM__C': 10, 'SVM__gamma': 0.5, 'pca__n_components': 20}
score for 8 fold CV -  0.80
 
 
{'SVM__C': 30, 'SVM__gamma': 0.07, 'pca__n_components': 20}
score for 9 fold CV -  0.82
 
 
{'SVM__C': 10, 'SVM__gamma': 0.5, 'pca__n_components': 20}
score for 10 fold CV -  0.82
 
 
{'SVM__C': 10, 'SVM__gamma': 0.5, 'pca__n_components': 20}


In [88]:
svmwpca2 = SVC(C = 30, gamma = 0.07)
svmwpca2.fit(x_train_pca, y_Train)
svmwpca2.score(x_test_pca, y_Test)

0.7268292682926829